In [1]:
import json
import random
import numpy
from gensim.models.fasttext import load_facebook_model
from nltk.stem import WordNetLemmatizer
from keras.utils import to_categorical
from nltk.tokenize import TweetTokenizer
from nltk import tokenize

Using TensorFlow backend.


In [9]:
trained_model = None
fasttext_model = None
lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()

In [19]:
def load_fasttext():
    global fasttext_model
    fasttext_model = load_facebook_model('wiki-news-300d-1M-subword.bin')


In [49]:
def fasttext_Vec(body):
    global fasttext_model
    tokens = tokenizer.tokenize(body)
    output = numpy.zeros(300)
    for token in tokens:
        try:
            output = numpy.add(output, fasttext_model[token.lower()])
        except Exception as e:
            print(e)
            output = numpy.add(output, numpy.zeros(300))
    return output

In [51]:
def classify():
    
    patients = []
    import csv

    with open('data.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            #print(f'\t{row["name"]} works in the {row["department"]} department, and was born in {row["birthday month"]}.')
            patient = []
            # process the input

            # patientID
            patient.append(row['PatientID'])

            patient.append(row['Patient Acct #'])

            #patient.append(row['First Name'])
            #patient.append(row['Last Name'])
            patient.append(row['First Name'] + ' ' + row['MI'] + ' ' + row['Last Name'])

            patient.append(row['Date of Birth'])

            patient.append(row['Current Street 1'] + ' ' + row['Current Street 2'] + ' ' + row['Current City'] + ' ' + row['Current State'] + ' ' + row['Current Zip Code'])
            
            patient.append(row['Previous First Name'] + ' ' + row['Previous MI'] + ' ' + row['Previous Last Name'])

            patient.append(row['Previous Street 1'] + ' ' + row['Previous Street 2'] + ' ' + row['Previous City'] + ' ' + row['Previous State'] + ' ' + row['Previous Zip Code'])

            patients.append(patient)

            line_count += 1
        print(f'Processed {line_count} lines.')
    
    return patients

In [20]:
load_fasttext()

In [50]:
fasttext_Vec("hello")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


array([-1.58778857e-02,  2.30641589e-02,  3.15905996e-02,  2.45315544e-02,
        1.89349670e-02, -1.65626183e-02,  1.36937127e-02, -4.63614166e-02,
       -3.76073830e-02, -1.68398824e-02, -1.62046459e-02,  3.15559544e-02,
        1.42231910e-02,  2.21537650e-02, -6.57501072e-03,  1.52964098e-02,
        4.37352285e-02,  4.55485657e-02,  2.19480656e-02, -4.06031646e-02,
       -2.13199966e-02, -1.00188358e-02, -5.01489155e-02,  2.93348618e-02,
        2.02660132e-02, -2.12313654e-03,  1.39874667e-02,  3.74888703e-02,
        3.36791873e-02, -2.49912776e-02,  9.68249864e-04,  4.45351079e-02,
        1.45235583e-02,  7.23537663e-03, -5.64728584e-03, -2.85669900e-02,
        3.28504406e-02,  2.42715217e-02,  1.16295552e-04, -1.46272480e-02,
       -3.35383713e-02, -8.59307498e-02, -2.36208411e-03, -1.00664664e-02,
        1.53780999e-02,  2.32233405e-02,  7.00091571e-03, -3.72130945e-02,
        2.57281354e-03,  5.74873500e-02,  8.05174559e-03,  7.17989635e-03,
        4.99734767e-02, -

In [22]:
from sklearn.metrics.pairwise import cosine_similarity


In [52]:
patients = classify()

Column names are GroupID, PatientID, Patient Acct #, First Name, MI, Last Name, Date of Birth, Sex, Current Street 1, Current Street 2, Current City, Current State, Current Zip Code, Previous First Name, Previous MI, Previous Last Name, Previous Street 1, Previous Street 2, Previous City, Previous State, Previous Zip Code
Processed 202 lines.


In [64]:
patients[0][4]


'1858 Sullivan Parkway  Fresno California 93726'

In [65]:
patients[4][4]


'1858 SULLIVAN PKWAY APT 2 FRESNO California 93726'

In [69]:
patients[10][4]

'715 Jana Point  Apache Junction Arizona 85219'

In [66]:
a = fasttext_Vec(patients[0][4])
b = fasttext_Vec(patients[4][4])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [67]:
aa = a.reshape(1,300)
ba = b.reshape(1,300)
cos_lib = cosine_similarity(aa, ba)

In [68]:
cos_lib

array([[0.56431365]])

In [70]:
c = fasttext_Vec(patients[10][4])
cc = c.reshape(1,300)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [74]:
cos_lib = cosine_similarity(aa, cc)
cos_lib

array([[0.7020264]])

In [78]:
a = fasttext_model['cam']
b = fasttext_model['cammy']
c = fasttext_model['robert']

aa = a.reshape(1,300)
bb = b.reshape(1,300)
cc = c.reshape(1,300)

cos_lib = cosine_similarity(aa, bb)
print(cos_lib)

cos_lib = cosine_similarity(cc, bb)
print(cos_lib)


[[0.50665134]]
[[0.4925518]]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
from fuzzywuzzy import fuzz

In [80]:
from fuzzywuzzy import process

In [96]:
fuzz.token_sort_ratio(patients[0][2], patients[4][2])

92